## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Seoul,37.5683,126.9778,45.84,51,27,1.86,KR,2021-10-24 22:09:14
1,1,Kruisfontein,-34.0033,24.7314,49.89,81,0,3.11,ZA,2021-10-24 22:07:08
2,2,Bonthe,7.5264,-12.5050,78.22,86,100,5.30,SL,2021-10-24 22:10:11
3,3,Port Hawkesbury,45.6169,-61.3485,48.58,83,90,8.01,CA,2021-10-24 22:10:11
4,4,Kodiak,57.7900,-152.4072,45.91,76,90,5.75,US,2021-10-24 22:09:20


In [26]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Bonthe,7.5264,-12.5050,78.22,86,100,5.30,SL,2021-10-24 22:10:11
5,5,Busselton,-33.6500,115.3333,55.04,82,100,9.91,AU,2021-10-24 22:10:12
6,6,Waingapu,-9.6567,120.2641,78.64,84,79,1.48,ID,2021-10-24 22:05:15
7,7,Saint-Philippe,-21.3585,55.7679,74.48,83,47,16.37,RE,2021-10-24 22:10:13
10,10,Kangaba,11.9333,-8.4167,79.75,65,78,2.84,ML,2021-10-24 22:10:14
11,11,Avarua,-21.2078,-159.7750,77.05,61,40,13.80,CK,2021-10-24 22:10:14
13,13,Victoria,22.2855,114.1577,70.25,73,37,1.99,HK,2021-10-24 22:08:40
14,14,Itaituba,-4.2761,-55.9836,84.65,61,99,2.75,BR,2021-10-24 22:10:15
15,15,Atuona,-9.8000,-139.0333,77.92,73,2,17.36,PF,2021-10-24 22:10:16
19,19,Mogadishu,2.0371,45.3438,79.43,81,45,13.76,SO,2021-10-24 22:10:17


In [29]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       4
Date          0
dtype: int64

In [56]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Bonthe,7.5264,-12.5050,78.22,86,100,5.30,SL,2021-10-24 22:10:11
5,5,Busselton,-33.6500,115.3333,55.04,82,100,9.91,AU,2021-10-24 22:10:12
6,6,Waingapu,-9.6567,120.2641,78.64,84,79,1.48,ID,2021-10-24 22:05:15
7,7,Saint-Philippe,-21.3585,55.7679,74.48,83,47,16.37,RE,2021-10-24 22:10:13
10,10,Kangaba,11.9333,-8.4167,79.75,65,78,2.84,ML,2021-10-24 22:10:14
...,...,...,...,...,...,...,...,...,...,...
660,660,Herat,34.5000,62.0000,53.47,22,0,5.75,AF,2021-10-24 22:14:59
662,662,Santarem,-2.4431,-54.7083,80.73,89,20,2.30,BR,2021-10-24 22:15:00
663,663,Simao,22.7886,100.9748,60.24,100,100,1.83,CN,2021-10-24 22:15:01
664,664,Siparia,10.1333,-61.5000,81.07,89,100,11.32,TT,2021-10-24 22:15:02


In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Bonthe,SL,78.22,7.5264,-12.5050,
5,Busselton,AU,55.04,-33.6500,115.3333,
6,Waingapu,ID,78.64,-9.6567,120.2641,
7,Saint-Philippe,RE,74.48,-21.3585,55.7679,
10,Kangaba,ML,79.75,11.9333,-8.4167,
11,Avarua,CK,77.05,-21.2078,-159.7750,
13,Victoria,HK,70.25,22.2855,114.1577,
14,Itaituba,BR,84.65,-4.2761,-55.9836,
15,Atuona,PF,77.92,-9.8000,-139.0333,
19,Mogadishu,SO,79.43,2.0371,45.3438,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
    #hotel_df.loc[index, "Hotel Name"] = 
        print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")     
        

Bonthe Holiday Village
Observatory Guest House
Pondok Wisata Elim
Chambres d'hôte "La Trinité"
CAF (case de l´amitié et de la fraternité)
Paradise Inn
Mini Hotel Central
Hotel Campos
Villa Enata
Hotel Juba
Hotel Xaguate
Keleti Beach Resort
Ellie's Villa
Toyoko Inn Iwaki Ekimae
Premier Inn Margate hotel
People ThankYou
Hampton Inn & Suites Jamestown
Mantra on Collins Hobart
Bora-Bora Pearl Beach Resort & Spa
Finch Bay Galapagos Hotel
Shwe Pyi Thar
Hotel América
Tu Casa
Hilo Hawaiian Hotel
The Halyards Hotel
Southern Sun Waterfront Cape Town
Pousada dos Navegantes
Wedgewood Motel
Blue Bay Lodge
Hotel Potosí Los Mirtos
Six Acres Bed & Breakfast
The Esplanade Resort & Spa
39 On Nile Guest House
Holiday Inn Express & Suites North Platte, an IHG Hotel
Calabash Hotel, Migori
Hotel not found... skipping.
Bluff Homestead - Guesthouse & Campervan Park
Royal Exchange Hotel
Shanti Maurice Resort & Spa
Lusaka Hotel
Butterfly Lodge
Sheraton Kauai Coconut Beach Resort
Isles Sunset Lodge
Hyōsetsu-sō
H

Montana Hotel Syariah
Undulata Country Lodge
Villa Las Tronas Hotel & Spa
Highlander Hotel & Apartments
فندق حضرموت
Holiday Inn Express Antofagasta, an IHG Hotel
Hotel not found... skipping.
Xiushui Hotel
GreenTree Hotel Rongcheng Bus Station
Bushido Lodge
Hotel La Villa Couacou
Westland Hotel AS
Auberge Madeli
B&B La Casona de Nazaret
Hotel Morotin
Oota Ryokan
Victorian Inn Beatrice
Área de Lazer Olho D'água
Pousada Guará's
Hotel not found... skipping.
فندق الفردوس
DoubleTree by Hilton Hotel Cleveland East Beachwood
Dibbo areaمنطقة دبو
Nordre Silda.
مهمان سرای شرکت گاز
Hotel Bressan
Hotel Pousada Tropical
Locanda Borgo Antico
The Green Dragon Hotel
Hotel Galvarino
صاحبة الفخامة
Sylvia Hotel
Parra Hotel & Suites
Motel 6 Hammond, IN - Chicago Area
Aryaduta Hotel
Camp Nirvana Yamunotri
Kost Pria
Aboiha Cottage
The Freeport Inn and Marina
Cozumel Palace® All Inclusive Resort
Fakhamat Al Orjoana Apartment
Takalvan Motel
Erlyn’s Sta Rosa City Home
عطر و ادکلن پارسیس
back to back rv park
Est

In [54]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotels_df =hotel_df.dropna()
clean_hotels_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Bonthe,SL,78.22,7.5264,-12.5050,Bonthe Holiday Village
5,Busselton,AU,55.04,-33.6500,115.3333,Observatory Guest House
6,Waingapu,ID,78.64,-9.6567,120.2641,Pondok Wisata Elim
7,Saint-Philippe,RE,74.48,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
10,Kangaba,ML,79.75,11.9333,-8.4167,CAF (case de l´amitié et de la fraternité)
...,...,...,...,...,...,...
660,Herat,AF,53.47,34.5000,62.0000,
662,Santarem,BR,80.73,-2.4431,-54.7083,Ecotur Lodge
663,Simao,CN,60.24,22.7886,100.9748,Changyu Hotel
664,Siparia,TT,81.07,10.1333,-61.5000,Brook Gardens


In [57]:
# 8a. Create the output File (CSV)
# Create CSV file of DF
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))